In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

In [2]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None
        
        new_row = pd.DataFrame({
                "account": ["WUSA"],
                "date": [time_now],
                "caption": str(event["description"])+" [For More Information, Click View Post] ",
                "display_photo": event["image"],
                "url": [event['url']],
                "likes": [0],
                "event_details": [{
                    "is_event": True,
                    "event_name": event["name"],
                    "event_description": str(event["description"])+" ... ",
                    "categories": ["Social"],
                    "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()),
                    "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()),
                    "location": location,
                }]
        })

        postsDf = pd.concat([postsDf, new_row])

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

'location'
'streetAddress'
'streetAddress'
'streetAddress'
  account        date                                            caption  \
0    WUSA  1725330397  📢 Hey WUSA Members! If this is your first time...   
0    WUSA  1725330397  CIBC x WUSA x UWaterloo presents Dillon Franci...   
0    WUSA  1725330397  Welcome Week 🍂 Get ready to FALL back into fun...   
0    WUSA  1725330397  Rise and shine, Warriors! WUSA’s inviting you ...   
0    WUSA  1725330397  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725330397  😍 A Welcome Week favourite is back! Are you re...   
0    WUSA  1725330397  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725330397  To give you a warm welcome back to campus, we ...   
0    WUSA  1725330397  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725330397  Don\'t miss out on the ultimate bowling experi...   
0    WUSA  1725330397  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725330397  The Ca

In [3]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with these IDs
result = collection.delete_many({'account': "WUSA"})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 21


In [4]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(1)


Inserted document ID: 66d673e879ba601bff9a567e
Inserted document ID: 66d673ea79ba601bff9a567f
Inserted document ID: 66d673eb79ba601bff9a5680
Inserted document ID: 66d673ec79ba601bff9a5681
Inserted document ID: 66d673ee79ba601bff9a5682
Inserted document ID: 66d673ef79ba601bff9a5683
Inserted document ID: 66d673f079ba601bff9a5684
Inserted document ID: 66d673f279ba601bff9a5685
Inserted document ID: 66d673f379ba601bff9a5686
Inserted document ID: 66d673f479ba601bff9a5687
Inserted document ID: 66d673f679ba601bff9a5688
Inserted document ID: 66d673f779ba601bff9a5689
Inserted document ID: 66d673f879ba601bff9a568a
Inserted document ID: 66d673f979ba601bff9a568b
Inserted document ID: 66d673fb79ba601bff9a568c
Inserted document ID: 66d673fc79ba601bff9a568d
Inserted document ID: 66d673fd79ba601bff9a568e
Inserted document ID: 66d673ff79ba601bff9a568f
Inserted document ID: 66d6740079ba601bff9a5690
Inserted document ID: 66d6740179ba601bff9a5691
Inserted document ID: 66d6740379ba601bff9a5692
